In [1]:
# There 2,000 images  dataset available on Kaggle, which contains 25,000 images.
# Here, we use a subset of the full dataset to decrease training time for Tutorial only!

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


--2020-09-07 14:23:37--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.140.128, 108.177.15.128, 173.194.76.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.140.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   100MB/s    in 0.7s    

2020-09-07 14:23:38 (100 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our Testing cat pictures
Testing_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our Testing dog pictures
Testing_dogs_dir = os.path.join(validation_dir, 'dogs')

In [3]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total Testing cat images:', len(os.listdir(Testing_cats_dir)))
print('total Testing dog images:', len(os.listdir(Testing_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total Testing cat images: 500
total Testing dog images: 500


In [4]:
from tensorflow.keras import layers
from tensorflow.keras import Model

# convolution/maxpooling  Layer - 1
img_input = layers.Input(shape=(150, 150, 3))
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# convolution/maxpooling layer Layer - 2
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# convolution/maxpooling  Layer - 3
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)


x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)

In [5]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)       

In [6]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [7]:
#ImagedataGenerator to prevent overfitting

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
100/100 - 10s - loss: 0.7067 - acc: 0.5755 - val_loss: 0.6080 - val_acc: 0.6610
Epoch 2/15
100/100 - 10s - loss: 0.5916 - acc: 0.6985 - val_loss: 0.5734 - val_acc: 0.7100
Epoch 3/15
100/100 - 10s - loss: 0.4998 - acc: 0.7555 - val_loss: 0.5688 - val_acc: 0.7080
Epoch 4/15
100/100 - 10s - loss: 0.4196 - acc: 0.8055 - val_loss: 0.5458 - val_acc: 0.7350
Epoch 5/15
100/100 - 10s - loss: 0.3505 - acc: 0.8370 - val_loss: 0.6335 - val_acc: 0.7290
Epoch 6/15
100/100 - 10s - loss: 0.2558 - acc: 0.8940 - val_loss: 0.6702 - val_acc: 0.7260
Epoch 7/15
100/100 - 10s - loss: 0.1980 - acc: 0.9195 - val_loss: 0.7288 - val_acc: 0.7330
Epoch 8/15
100/100 - 10s - loss: 0.1407 - acc: 0.9475 - val_loss: 0.8403 - val_acc: 0.7190
Epoch 9/15
100/100 - 10s - loss: 0.1215 - acc: 0.9620 - val_loss: 1.0623 - val_acc: 0.7220
Epoch 10/15
100/100 - 10s - loss: 0.0965 - acc: 0.9720 - val_loss: 1.1336 - val_acc: 0.7120
Epoch 11/15
1

In [10]:
#get acess google drive data into google colab

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('finish')

finish


In [13]:

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/dataset/single_prediction/cat_or_dog_1.jpg', target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)
train_generator.class_indices
if result[0][0] == 1:
  prediction = "Dog"
  print(prediction)
else:
  prediction = "Cat"
  print(prediction)

Dog


In [14]:
test_image = image.load_img('/content/dataset/single_prediction/cat_or_dog_2.jpg', target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)
train_generator.class_indices
if result[0][0] == 1:
  prediction = "Dog"
  print(prediction)
else:
  prediction = "Cat"
  print(prediction)

Cat
